# Imports

In [58]:
import pandas as pd
import numpy as np

# Use Dataset

In [59]:
df = pd.read_csv('dataset/mobile_addiction_data.csv')

df.head()

,User_ID,Country,Age,Gender,Occupation,Education_Level,Income_USD,Daily_Screen_Time_Hours,Phone_Unlocks_Per_Day,Social_Media_Usage_Hours,Gaming_Usage_Hours,Streaming_Usage_Hours,Messaging_Usage_Hours,Work_Related_Usage_Hours,Sleep_Hours,Physical_Activity_Hours,Mental_Health_Score,Depression_Score,Anxiety_Score,Stress_Level,Relationship_Status,Has_Children,Urban_or_Rural,Time_Spent_With_Family_Hours,Online_Shopping_Hours,Internet_Connection_Type,Primary_Device_Brand,Has_Screen_Time_Management_App,Self_Reported_Addiction_Level,Monthly_Data_Usage_GB,Has_Night_Mode_On,Age_First_Phone,Push_Notifications_Per_Day,Tech_Savviness_Score
0,1,India,32,Male,Salesperson,High School,43865.49,5.81,75,0.84,-1.55,3.36,1.44,2.14,5.22,1.32,61.47,80.80,23.72,95.80,Single,No,Rural,1.70,1.85,5G,Other,No,Low,16.43,Yes,16,106,39.36
1,2,UK,26,Male,Artist,Master's,41868.19,9.05,61,3.13,2.50,0.80,2.42,1.44,6.23,0.66,96.11,98.99,39.72,25.03,In Relationship,No,Rural,0.90,0.66,4G,Samsung,Yes,Severe,32.87,No,12,111,9.45
2,3,Germany,70,Other,Doctor,High School,59636.51,5.76,58,2.12,1.96,1.84,1.98,1.92,6.57,1.93,73.68,44.64,94.47,99.54,Single,Yes,Urban,1.69,-0.14,WiFi,Samsung,Yes,Severe,27.45,No,18,90,50.27
3,4,UK,44,Female,Engineer,NaN,39022.07,6.71,80,1.60,2.70,1.90,2.43,1.82,7.27,0.94,53.42,10.89,5.03,66.49,In Relationship,No,Urban,1.11,0.17,3G,Apple,Yes,Moderate,30.85,No,17,60,30.82
4,5,Brazil,46,Other,Student,NaN,-783.98,6.31,136,1.51,1.73,1.48,1.30,1.76,5.43,1.04,67.01,37.54,17.29,14.72,Divorced,No,Urban,1.06,0.58,3G,Xiaomi,No,High,10.38,Yes,18,127,21.70


In [60]:
print("Number of rows::",df.shape[0])
print("Number of columns::",df.shape[1])
print ("Column Names &  Data Types::\n",df.dtypes)

Number of rows:: 3000
Number of columns:: 34
Column Names &  Data Types::
 User_ID                             int64
Country                            object
Age                                 int64
Gender                             object
Occupation                         object
Education_Level                    object
Income_USD                        float64
Daily_Screen_Time_Hours           float64
Phone_Unlocks_Per_Day               int64
Social_Media_Usage_Hours          float64
Gaming_Usage_Hours                float64
Streaming_Usage_Hours             float64
Messaging_Usage_Hours             float64
Work_Related_Usage_Hours          float64
Sleep_Hours                       float64
Physical_Activity_Hours           float64
Mental_Health_Score               float64
Depression_Score                  float64
Anxiety_Score                     float64
Stress_Level                      float64
Relationship_Status                object
Has_Children                       object
U

In [50]:
print("Columns with Missing Values::", df.columns[df.isnull().any()].tolist())
print("Number of rows with Missing Values::", df.isnull().any(axis=1).sum())
print("Sample Indices with missing data::", df.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()[0:10])

Columns with Missing Values:: ['Education_Level']
Number of rows with Missing Values:: 612
Sample Indices with missing data:: [3, 4, 8, 16, 18, 40, 44, 45, 55, 66]


In [67]:
print("General Statistics::\n")
print(df.info())

print("Summary Statistics::\n")
print(df.describe(include='all'))

General Statistics::

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        3000 non-null   object 
 1   Age                            3000 non-null   int64  
 2   Gender                         3000 non-null   object 
 3   Occupation                     3000 non-null   object 
 4   Education_Level                2388 non-null   object 
 5   Income_USD                     3000 non-null   float64
 6   Daily_Screen_Time_Hours        3000 non-null   float64
 7   Phone_Unlocks_Per_Day          3000 non-null   int64  
 8   Social_Media_Usage_Hours       3000 non-null   float64
 9   Gaming_Usage_Hours             3000 non-null   float64
 10  Streaming_Usage_Hours          3000 non-null   float64
 11  Messaging_Usage_Hours          3000 non-null   float64
 12  Work_Related_Usage_Hours  

In [62]:
# drop unnecessary columns
df = df.drop(columns=[
    'User_ID',
    'Primary_Device_Brand',
    'Internet_Connection_Type',
    'Has_Screen_Time_Management_App',
    'Has_Night_Mode_On'
], errors='ignore')

df.head()

,Country,Age,Gender,Occupation,Education_Level,Income_USD,Daily_Screen_Time_Hours,Phone_Unlocks_Per_Day,Social_Media_Usage_Hours,Gaming_Usage_Hours,Streaming_Usage_Hours,Messaging_Usage_Hours,Work_Related_Usage_Hours,Sleep_Hours,Physical_Activity_Hours,Mental_Health_Score,Depression_Score,Anxiety_Score,Stress_Level,Relationship_Status,Has_Children,Urban_or_Rural,Time_Spent_With_Family_Hours,Online_Shopping_Hours,Self_Reported_Addiction_Level,Monthly_Data_Usage_GB,Age_First_Phone,Push_Notifications_Per_Day,Tech_Savviness_Score
0,India,32,Male,Salesperson,High School,43865.49,5.81,75,0.84,-1.55,3.36,1.44,2.14,5.22,1.32,61.47,80.80,23.72,95.80,Single,No,Rural,1.70,1.85,Low,16.43,16,106,39.36
1,UK,26,Male,Artist,Master's,41868.19,9.05,61,3.13,2.50,0.80,2.42,1.44,6.23,0.66,96.11,98.99,39.72,25.03,In Relationship,No,Rural,0.90,0.66,Severe,32.87,12,111,9.45
2,Germany,70,Other,Doctor,High School,59636.51,5.76,58,2.12,1.96,1.84,1.98,1.92,6.57,1.93,73.68,44.64,94.47,99.54,Single,Yes,Urban,1.69,-0.14,Severe,27.45,18,90,50.27
3,UK,44,Female,Engineer,NaN,39022.07,6.71,80,1.60,2.70,1.90,2.43,1.82,7.27,0.94,53.42,10.89,5.03,66.49,In Relationship,No,Urban,1.11,0.17,Moderate,30.85,17,60,30.82
4,Brazil,46,Other,Student,NaN,-783.98,6.31,136,1.51,1.73,1.48,1.30,1.76,5.43,1.04,67.01,37.54,17.29,14.72,Divorced,No,Urban,1.06,0.58,High,10.38,18,127,21.70


In [63]:
numeric_df = df.select_dtypes(include=[np.number])

negatives_mask = numeric_df < 0

negatives_count = negatives_mask.sum()

print("=== Negative Number Summary ===")
for col, count in negatives_count.items():
    if count > 0:
        print(f"{col}: {count} negative values")
    else:
        print(f"{col}: No negative values")

=== Negative Number Summary ===
Age: No negative values
Income_USD: 81 negative values
Daily_Screen_Time_Hours: 9 negative values
Phone_Unlocks_Per_Day: 1 negative values
Social_Media_Usage_Hours: 70 negative values
Gaming_Usage_Hours: 213 negative values
Streaming_Usage_Hours: 67 negative values
Messaging_Usage_Hours: 2 negative values
Work_Related_Usage_Hours: 53 negative values
Sleep_Hours: No negative values
Physical_Activity_Hours: 80 negative values
Mental_Health_Score: No negative values
Depression_Score: No negative values
Anxiety_Score: No negative values
Stress_Level: No negative values
Time_Spent_With_Family_Hours: 96 negative values
Online_Shopping_Hours: 181 negative values
Monthly_Data_Usage_GB: 16 negative values
Age_First_Phone: No negative values
Push_Notifications_Per_Day: 3 negative values
Tech_Savviness_Score: No negative values


In [64]:
# show rows that contain any negative numeric values
rows_with_negatives = df[negatives_mask.any(axis=1)]

rows_with_negatives.head()

,Country,Age,Gender,Occupation,Education_Level,Income_USD,Daily_Screen_Time_Hours,Phone_Unlocks_Per_Day,Social_Media_Usage_Hours,Gaming_Usage_Hours,Streaming_Usage_Hours,Messaging_Usage_Hours,Work_Related_Usage_Hours,Sleep_Hours,Physical_Activity_Hours,Mental_Health_Score,Depression_Score,Anxiety_Score,Stress_Level,Relationship_Status,Has_Children,Urban_or_Rural,Time_Spent_With_Family_Hours,Online_Shopping_Hours,Self_Reported_Addiction_Level,Monthly_Data_Usage_GB,Age_First_Phone,Push_Notifications_Per_Day,Tech_Savviness_Score
0,India,32,Male,Salesperson,High School,43865.49,5.81,75,0.84,-1.55,3.36,1.44,2.14,5.22,1.32,61.47,80.80,23.72,95.80,Single,No,Rural,1.70,1.85,Low,16.43,16,106,39.36
2,Germany,70,Other,Doctor,High School,59636.51,5.76,58,2.12,1.96,1.84,1.98,1.92,6.57,1.93,73.68,44.64,94.47,99.54,Single,Yes,Urban,1.69,-0.14,Severe,27.45,18,90,50.27
4,Brazil,46,Other,Student,NaN,-783.98,6.31,136,1.51,1.73,1.48,1.30,1.76,5.43,1.04,67.01,37.54,17.29,14.72,Divorced,No,Urban,1.06,0.58,High,10.38,18,127,21.70
12,USA,18,Female,Engineer,Bachelor's,16430.69,5.01,75,1.12,0.69,1.26,1.63,1.11,7.42,1.40,88.70,58.26,24.99,23.15,Divorced,Yes,Urban,2.16,-0.23,Severe,29.76,8,112,37.84
17,Mexico,18,Male,Doctor,PhD,40513.10,6.06,68,1.38,-0.52,1.27,1.99,2.46,6.68,1.07,61.56,51.64,48.92,41.25,In Relationship,No,Urban,2.33,0.90,Severe,31.20,13,12,47.86


In [65]:
# replace all negative hours to 0
time_columns = [
    "Daily_Screen_Time_Hours",
    "Phone_Unlocks_Per_Day",
    "Social_Media_Usage_Hours",
    "Gaming_Usage_Hours",
    "Streaming_Usage_Hours",
    "Messaging_Usage_Hours",
    "Work_Related_Usage_Hours",
    "Sleep_Hours",
    "Physical_Activity_Hours",
    "Time_Spent_With_Family_Hours",
    "Online_Shopping_Hours",
    "Monthly_Data_Usage_GB",
    "Push_Notifications_Per_Day"
]

df[time_columns] = df[time_columns].clip(lower=0)

rows_with_negatives = df[negatives_mask.any(axis=1)]

rows_with_negatives.head()

,Country,Age,Gender,Occupation,Education_Level,Income_USD,Daily_Screen_Time_Hours,Phone_Unlocks_Per_Day,Social_Media_Usage_Hours,Gaming_Usage_Hours,Streaming_Usage_Hours,Messaging_Usage_Hours,Work_Related_Usage_Hours,Sleep_Hours,Physical_Activity_Hours,Mental_Health_Score,Depression_Score,Anxiety_Score,Stress_Level,Relationship_Status,Has_Children,Urban_or_Rural,Time_Spent_With_Family_Hours,Online_Shopping_Hours,Self_Reported_Addiction_Level,Monthly_Data_Usage_GB,Age_First_Phone,Push_Notifications_Per_Day,Tech_Savviness_Score
0,India,32,Male,Salesperson,High School,43865.49,5.81,75,0.84,0.00,3.36,1.44,2.14,5.22,1.32,61.47,80.80,23.72,95.80,Single,No,Rural,1.70,1.85,Low,16.43,16,106,39.36
2,Germany,70,Other,Doctor,High School,59636.51,5.76,58,2.12,1.96,1.84,1.98,1.92,6.57,1.93,73.68,44.64,94.47,99.54,Single,Yes,Urban,1.69,0.00,Severe,27.45,18,90,50.27
4,Brazil,46,Other,Student,NaN,-783.98,6.31,136,1.51,1.73,1.48,1.30,1.76,5.43,1.04,67.01,37.54,17.29,14.72,Divorced,No,Urban,1.06,0.58,High,10.38,18,127,21.70
12,USA,18,Female,Engineer,Bachelor's,16430.69,5.01,75,1.12,0.69,1.26,1.63,1.11,7.42,1.40,88.70,58.26,24.99,23.15,Divorced,Yes,Urban,2.16,0.00,Severe,29.76,8,112,37.84
17,Mexico,18,Male,Doctor,PhD,40513.10,6.06,68,1.38,0.00,1.27,1.99,2.46,6.68,1.07,61.56,51.64,48.92,41.25,In Relationship,No,Urban,2.33,0.90,Severe,31.20,13,12,47.86


In [66]:
# Select all rows that contains NaN value
nan_rows = df[df.isna().any(axis=1)]

print("=== Rows containing NaN values ===")
nan_rows.head()

=== Rows containing NaN values ===


,Country,Age,Gender,Occupation,Education_Level,Income_USD,Daily_Screen_Time_Hours,Phone_Unlocks_Per_Day,Social_Media_Usage_Hours,Gaming_Usage_Hours,Streaming_Usage_Hours,Messaging_Usage_Hours,Work_Related_Usage_Hours,Sleep_Hours,Physical_Activity_Hours,Mental_Health_Score,Depression_Score,Anxiety_Score,Stress_Level,Relationship_Status,Has_Children,Urban_or_Rural,Time_Spent_With_Family_Hours,Online_Shopping_Hours,Self_Reported_Addiction_Level,Monthly_Data_Usage_GB,Age_First_Phone,Push_Notifications_Per_Day,Tech_Savviness_Score
3,UK,44,Female,Engineer,NaN,39022.07,6.71,80,1.60,2.70,1.90,2.43,1.82,7.27,0.94,53.42,10.89,5.03,66.49,In Relationship,No,Urban,1.11,0.17,Moderate,30.85,17,60,30.82
4,Brazil,46,Other,Student,NaN,-783.98,6.31,136,1.51,1.73,1.48,1.30,1.76,5.43,1.04,67.01,37.54,17.29,14.72,Divorced,No,Urban,1.06,0.58,High,10.38,18,127,21.70
8,Nigeria,33,Other,Salesperson,NaN,26629.11,7.45,50,2.02,0.49,2.41,1.60,2.98,6.54,1.40,55.45,59.96,95.10,95.94,Divorced,No,Urban,0.77,0.10,High,24.49,8,85,31.52
16,USA,51,Other,Manager,NaN,41764.15,8.19,47,1.29,1.72,1.44,1.16,2.72,6.34,0.35,99.02,39.11,41.98,77.20,In Relationship,Yes,Rural,1.64,0.37,Moderate,23.55,12,149,10.65
18,USA,33,Male,Manager,NaN,6521.03,5.79,47,0.00,0.34,1.88,1.23,1.11,4.86,1.11,28.44,29.96,15.42,16.75,Married,Yes,Rural,0.00,0.77,Severe,41.45,11,99,14.92
